In [1]:
import audb
import pandas as pd
import os
import glob
import re

In [ ]:
datasets = audb.available()

targets = ["ravdess", "crema-d", "tess", "savee"]

datasets[datasets.index.isin(targets)].sort_values(by=["name", "version"], ascending=[True, False])

In [ ]:
class audb_dataset:
    def __init__(self, name: str, version: str | None = None):
        self.name = name
        self.version = version
        
crema_d = audb_dataset(name = "crema-d", version = "1.3.0")
ravdess = audb_dataset(name = "ravdess", version = "1.1.3")


#### CREMA-D

In [ ]:
crema_db = audb.load(crema_d.name,
                    version = crema_d.version,
                    format = "wav", 
                    sampling_rate= 16000,
                    mixdown = True,
                    pickle_tables= False,
                    cache_root="./"
)

In [ ]:
crema_files = pd.read_csv('crema-d/1.3.0/fe182b91/db.files.csv')
crema_files

In [ ]:
crema_sentences = pd.read_csv('crema-d/1.3.0/fe182b91/db.sentence.csv')
crema_sentences

In [ ]:
crema_complete_df = crema_files.merge(crema_sentences, on = "file", how = "left")
crema_complete_df

In [ ]:
crema_complete_df = crema_complete_df[crema_complete_df["corrupted"] == False]
crema_complete_df.shape

In [ ]:
EMOTION_LABELS = {
    "ANG": "anger",
    "DIS": "disgust",
    "FEA": "fear",
    "HAP": "happy/joy",
    "NEU": "neutral",
    "SAD": "sad"
}

SENTENCE_LABELS = {
    "IEO": "It's eleven o'clock",
    "TIE": "That is exactly what happened",
    "IOM": "I'm on my way to the meeting",
    "IWW": "I wonder what this is about",
    "TAI": "The airplane is almost full",
    "MTI": "Maybe tomorrow it will be cold",
    "IWL": "I would like a new alarm clock",
    "ITH": "I think I have a doctor's appointment",
    "DFA": "Don't forget a jacket",
    "ITS": "I think I've seen this before",
    "TSI": "The surface is slick",
    "WSI": "We'll stop in a couple of minutes"
}
        

In [ ]:
example = "1091/1091_WSI_DIS_XX.wav"
example.replace("/", "_").replace(".", "_").split("_")[3]

def extract_emotion_code(file_path: str) -> str:
    return file_path.replace("/", "_").replace(".", "_").split("_")[3]

In [ ]:
crema_complete_df["emotion_code"] = crema_complete_df["file"].apply(extract_emotion_code)
crema_complete_df["emotion"] = crema_complete_df["emotion_code"].map(EMOTION_LABELS)
crema_complete_df

In [ ]:
crema_complete_df.rename(columns={"sentence": "sentence_code"}, inplace=True)
crema_complete_df["sentence"] = crema_complete_df["sentence_code"].map(SENTENCE_LABELS)
crema_complete_df

In [ ]:
columns_order = ["file", "speaker", "corrupted", "sentence_code", "sentence", "emotion_code", "emotion"]
crema_complete_df = crema_complete_df[columns_order]
crema_complete_df

In [ ]:
crema_complete_df.to_csv("CREMA_D/crema_complete_files.csv", index = False)

#### RAVDESS

In [ ]:
ravdess_db = audb.load(ravdess.name,
                    version = ravdess.version,
                    format = "wav", 
                    sampling_rate= 16000,
                    mixdown = True,
                    pickle_tables= False,
                    cache_root="./"
)

In [ ]:
from fastparquet import ParquetFile as fp
from pathlib import Path

In [ ]:
ravdess_files = fp("RAVDESS/1.1.3/fe182b91/db.files.parquet").to_pandas()
ravdess_files

In [ ]:
db_folder = Path("RAVDESS/1.1.3/fe182b91")
db_files = list(db_folder.glob("*.parquet"))

In [ ]:
emotions_df = pd.DataFrame()
for path in db_files[1:]:
    
    try:
        df = fp(path).to_pandas()
        emotions_df = pd.concat([emotions_df, df], ignore_index=True)
        print(f"Concatenated {path.name} successfully.")

    except Exception as e:
        print(f"Failed to concatenate {path.name}: {e}")
        
emotions_df

In [ ]:
ravdess_complete_df = ravdess_files.merge(emotions_df, on = "file", how = "left")
ravdess_complete_df.drop(columns=["emotional intensity"], inplace=True)
ravdess_complete_df

In [ ]:
if os.path.exists("RAVDESS/ravdess_complete_files.csv"):
    print("File already exists. Skipping save to avoid overwriting.")
else:
    ravdess_complete_df.to_csv("RAVDESS/ravdess_complete_files.csv", index=False)
    print("File saved successfully.")


#### SAVEE

In [ ]:
savee_root = "SAVEE/data"

file_paths = glob.glob(os.path.join(savee_root, "*.wav"))

savee_complete_df = pd.DataFrame(file_paths, columns=["file"])
print(savee_complete_df.shape)
savee_complete_df

In [ ]:
EMOTION_MAPPING_SAVEE = {
    'a': 'anger', 
    'd': 'disgust',
    'f': 'fear',
    'h': 'happy',
    'n': 'neutral',
    'sa': 'sadness',
    'su': 'surprise'
}

pattern_savee = r"(?P<speaker>[a-zA-Z]+)_(?P<emotion>[a-z]+)\d+\.wav"

def ext_speaker(file_path: str) -> str:
    base_name = os.path.basename(file_path)
    match = re.match(pattern_savee, base_name)
    if match:
        return match.group("speaker")
    return "unknown"

def ext_emotion(file_path: str) -> str:
    base_name = os.path.basename(file_path)
    match = re.match(pattern_savee, base_name)
    if match:
        return EMOTION_MAPPING_SAVEE.get(match.group("emotion"), "unknown")
    return "unknown"


In [ ]:
savee_complete_df["emotion"] = savee_complete_df["file"].apply(ext_emotion)

savee_complete_df["speaker"] = savee_complete_df["file"].apply(ext_speaker)
savee_complete_df["source"] = "savee"
savee_complete_df

In [ ]:
if os.path.exists("SAVEE/savee_complete_files.csv"  ):
    print("File already exists. Skipping save to avoid overwriting.")
else:
    savee_complete_df.to_csv("SAVEE/savee_complete_files.csv", index = False)
    print("File saved successfully.")

#### TESS

In [ ]:
tess_complete_df = pd.read_csv("TESS/MANIFEST.TXT", sep=" ", header=None, names=["file", "unused1", "unused2", "unused3"])
tess_complete_df


In [ ]:
tess_complete_df.drop(columns=["unused1", "unused2", "unused3"], inplace=True)

tess_root = os.path.join("TESS", "data")

tess_complete_df["file"] = tess_complete_df["file"].apply(lambda x: os.path.join(tess_root, x))
tess_complete_df

In [ ]:
EMOTION_MAPPING_SAVEE = {
    'a': 'anger', 
    'd': 'disgust',
    'f': 'fear',
    'h': 'happy',
    'n': 'neutral',
    'sa': 'sadness',
    'su': 'surprise'
}

pattern_savee = r"(?P<speaker>[A-Z]+)_(?P<expression>[a-z]+)_(?P<emotion>[a-z]+)\.wav"

def ext_speaker(file_path: str) -> str:
    base_name = os.path.basename(file_path)
    match = re.match(pattern_savee, base_name)
    if match:
        return match.group("speaker")
    return "unknown"

def ext_emotion(file_path: str) -> str:
    base_name = os.path.basename(file_path)
    match = re.match(pattern_savee, base_name)
    if match:
        return match.group("emotion")
    return "unknown"



In [ ]:
tess_complete_df["speaker"] = tess_complete_df["file"].apply(ext_speaker)
tess_complete_df["emotion"] = tess_complete_df["file"].apply(ext_emotion)
tess_complete_df

In [ ]:
if os.path.exists("TESS/tess_complete_files.csv"  ):
    print("File already exists. Skipping save to avoid overwriting.")
else:
    tess_complete_df.to_csv("TESS/tess_complete_files.csv", index = False)
    print("File saved successfully.")

#### EMOITA

In [ ]:
datasets = audb.available()

targets = ["emozionalmente"]

datasets[datasets.index.isin(targets)].sort_values(by=["name", "version"], ascending=[True, False])

In [ ]:
class audb_dataset:
    def __init__(self, name: str, version: str | None = None):
        self.name = name
        self.version = version
        
emoita = audb_dataset(name = "emozionalmente", version = "1.0.0")

In [ ]:
emoita_db = audb.load(emoita.name,
                    version = emoita.version,
                    format = "wav", 
                    sampling_rate= 16000,
                    mixdown = True,
                    pickle_tables= False,
                    cache_root="./"
)

In [2]:
tmp_train = pd.read_csv('emozionalmente/1.0.0/fe182b91/db.emotion.categories.train.desired.csv')
tmp_train

,file,emotion
0,audio/1613671614352.wav,anger
1,audio/1613658275427.wav,anger
2,audio/1613324357435.wav,anger
3,audio/1614274086698.wav,anger
4,audio/1612982146424.wav,anger
...,...,...
3983,audio/1616505528024.wav,surprise
3984,audio/1614627012378.wav,surprise
3985,audio/1613310701161.wav,surprise
3986,audio/1613403675624.wav,surprise


In [3]:
tmp_train.emotion.value_counts()

emotion
happiness    588
neutral      588
surprise     571
fear         566
disgust      564
anger        559
sadness      552
Name: count, dtype: int64

In [4]:
emoita_raw_files = pd.read_csv('emozionalmente/1.0.0/fe182b91/db.files.csv')
emoita_raw_files

,file,speaker,transcription
0,audio/1614951631538.wav,d4bbc5043503b9aed309ede00854ee48937684b57a1cc0...,s0
1,audio/1613671614352.wav,b830cbf6e79aeeeec1fbf2ef3be741628ffc574c9b15f3...,s1
2,audio/1613994293936.wav,acdd987f9c1700b25898d3bd30c201df5ad4f34b3ca5eb...,s2
3,audio/1617135480050.wav,28043d2516f2d956b81ce66cc01fbd427ac54ff1eb3a07...,s0
4,audio/1613658275427.wav,5b8476bb94a4a1a17df32b417ec035029f48e4487c9055...,s3
...,...,...,...
6897,audio/1613310701161.wav,1ae2a40d6e3a1b8c6041da953f650dc017fea8c646648b...,s0
6898,audio/1613061195435.wav,f4d0c9178a221556efe45a4ea3d2bb77b470626409af7f...,s17
6899,audio/1615231434303.wav,7ce215563f7b620fdd32c6618a95215100b4fbede0eb4f...,s14
6900,audio/1613403675624.wav,1dab1d691c3cb60f475163ff79b4fa2f58bac807098245...,s4


In [5]:
emoita_train = tmp_train.merge(emoita_raw_files, on="file", how="left")
emoita_train.drop(columns=["transcription"], inplace=True)
emoita_train

,file,emotion,speaker
0,audio/1613671614352.wav,anger,b830cbf6e79aeeeec1fbf2ef3be741628ffc574c9b15f3...
1,audio/1613658275427.wav,anger,5b8476bb94a4a1a17df32b417ec035029f48e4487c9055...
2,audio/1613324357435.wav,anger,f78d765ec69ed2306cf544ae70b352f27ac00ced795ad5...
3,audio/1614274086698.wav,anger,33fae0227059f08853c11257fe73f4976d4527614588b9...
4,audio/1612982146424.wav,anger,b3c7c357c77848765caaded035be8800169fb4dc53996f...
...,...,...,...
3983,audio/1616505528024.wav,surprise,f78d765ec69ed2306cf544ae70b352f27ac00ced795ad5...
3984,audio/1614627012378.wav,surprise,8cd4289dc47d0ea3a3deca73495829e5c4a9c96b1c4616...
3985,audio/1613310701161.wav,surprise,1ae2a40d6e3a1b8c6041da953f650dc017fea8c646648b...
3986,audio/1613403675624.wav,surprise,1dab1d691c3cb60f475163ff79b4fa2f58bac807098245...


In [6]:
emoita_train.rename(columns={"file" : "old_filename"}, inplace=True)

emoita_root = os.path.join("EMOITA", "data")

emoita_train["file"] = emoita_train["old_filename"].apply(lambda x: os.path.join(emoita_root, os.path.basename(x)))
emoita_train



,old_filename,emotion,speaker,file
0,audio/1613671614352.wav,anger,b830cbf6e79aeeeec1fbf2ef3be741628ffc574c9b15f3...,EMOITA/data/1613671614352.wav
1,audio/1613658275427.wav,anger,5b8476bb94a4a1a17df32b417ec035029f48e4487c9055...,EMOITA/data/1613658275427.wav
2,audio/1613324357435.wav,anger,f78d765ec69ed2306cf544ae70b352f27ac00ced795ad5...,EMOITA/data/1613324357435.wav
3,audio/1614274086698.wav,anger,33fae0227059f08853c11257fe73f4976d4527614588b9...,EMOITA/data/1614274086698.wav
4,audio/1612982146424.wav,anger,b3c7c357c77848765caaded035be8800169fb4dc53996f...,EMOITA/data/1612982146424.wav
...,...,...,...,...
3983,audio/1616505528024.wav,surprise,f78d765ec69ed2306cf544ae70b352f27ac00ced795ad5...,EMOITA/data/1616505528024.wav
3984,audio/1614627012378.wav,surprise,8cd4289dc47d0ea3a3deca73495829e5c4a9c96b1c4616...,EMOITA/data/1614627012378.wav
3985,audio/1613310701161.wav,surprise,1ae2a40d6e3a1b8c6041da953f650dc017fea8c646648b...,EMOITA/data/1613310701161.wav
3986,audio/1613403675624.wav,surprise,1dab1d691c3cb60f475163ff79b4fa2f58bac807098245...,EMOITA/data/1613403675624.wav


In [11]:
emoita_speakers = pd.read_csv("emozionalmente/1.0.0/fe182b91/db.speaker.csv")
emoita_speakers

,speaker,age,gender,mother_tongue
0,f04bb0e6361c05acba4d5185a2d372177bdb77898c1a21...,30,female,italian
1,b395b3d82da20e930e20220a5ab4de9adb9a12aaa7cecf...,26,male,italian
2,69a6f8fa8a7d2e7338bddab3f872ba3532c914f62ebfeb...,26,female,italian
3,5b1d678a1b4936c172e8c239785f89a2693f7603890c41...,54,female,italian
4,1718aa7e2d0821e3ade5783b4e045ab5930ec22054a0fe...,11,female,italian
...,...,...,...,...
426,90faa565f21422b948417c0e69766d0df3331b1b347f88...,33,female,italian
427,faf6ce1ea46bca3ee95197b53c7137840d63e6caddc447...,27,male,italian
428,7da9f2113516c871e275194f8b2e7521f0c3fb77728ca9...,27,male,italian
429,39bf573469551ecb760d59bd0e38e18c2727456e62a732...,22,male,italian


In [12]:
emoita_speakers.iloc[0]['speaker']

'f04bb0e6361c05acba4d5185a2d372177bdb77898c1a213f5dfe5c1d60331ea1'

In [13]:
unique_hashes = emoita_speakers['speaker'].unique()

speaker_map = {original : f"speaker_{i+1:03d}" for i, original in enumerate(unique_hashes)}

emoita_speakers["speaker_new_id"] = emoita_speakers["speaker"].map(speaker_map)
emoita_speakers

,speaker,age,gender,mother_tongue,speaker_new_id
0,f04bb0e6361c05acba4d5185a2d372177bdb77898c1a21...,30,female,italian,speaker_001
1,b395b3d82da20e930e20220a5ab4de9adb9a12aaa7cecf...,26,male,italian,speaker_002
2,69a6f8fa8a7d2e7338bddab3f872ba3532c914f62ebfeb...,26,female,italian,speaker_003
3,5b1d678a1b4936c172e8c239785f89a2693f7603890c41...,54,female,italian,speaker_004
4,1718aa7e2d0821e3ade5783b4e045ab5930ec22054a0fe...,11,female,italian,speaker_005
...,...,...,...,...,...
426,90faa565f21422b948417c0e69766d0df3331b1b347f88...,33,female,italian,speaker_427
427,faf6ce1ea46bca3ee95197b53c7137840d63e6caddc447...,27,male,italian,speaker_428
428,7da9f2113516c871e275194f8b2e7521f0c3fb77728ca9...,27,male,italian,speaker_429
429,39bf573469551ecb760d59bd0e38e18c2727456e62a732...,22,male,italian,speaker_430


In [14]:
if os.path.exists("EMOITA/speakers.csv"):
    print("File already exists. Skipping save to avoid overwriting.")
else:
    emoita_speakers.to_csv("EMOITA/speakers.csv", index = False)
    print("File saved successfully.")

File saved successfully.


In [15]:
emoita_train.replace({"speaker": speaker_map}, inplace=True)
emoita_train

,old_filename,emotion,speaker,file
0,audio/1613671614352.wav,anger,speaker_321,EMOITA/data/1613671614352.wav
1,audio/1613658275427.wav,anger,speaker_303,EMOITA/data/1613658275427.wav
2,audio/1613324357435.wav,anger,speaker_314,EMOITA/data/1613324357435.wav
3,audio/1614274086698.wav,anger,speaker_109,EMOITA/data/1614274086698.wav
4,audio/1612982146424.wav,anger,speaker_179,EMOITA/data/1612982146424.wav
...,...,...,...,...
3983,audio/1616505528024.wav,surprise,speaker_314,EMOITA/data/1616505528024.wav
3984,audio/1614627012378.wav,surprise,speaker_079,EMOITA/data/1614627012378.wav
3985,audio/1613310701161.wav,surprise,speaker_206,EMOITA/data/1613310701161.wav
3986,audio/1613403675624.wav,surprise,speaker_258,EMOITA/data/1613403675624.wav


In [17]:
emoita_train = emoita_train[["file", "speaker", "emotion"]]
emoita_train["split"] = "train"
emoita_train

,file,speaker,emotion,split
0,EMOITA/data/1613671614352.wav,speaker_321,anger,train
1,EMOITA/data/1613658275427.wav,speaker_303,anger,train
2,EMOITA/data/1613324357435.wav,speaker_314,anger,train
3,EMOITA/data/1614274086698.wav,speaker_109,anger,train
4,EMOITA/data/1612982146424.wav,speaker_179,anger,train
...,...,...,...,...
3983,EMOITA/data/1616505528024.wav,speaker_314,surprise,train
3984,EMOITA/data/1614627012378.wav,speaker_079,surprise,train
3985,EMOITA/data/1613310701161.wav,speaker_206,surprise,train
3986,EMOITA/data/1613403675624.wav,speaker_258,surprise,train


In [18]:
if os.path.exists("EMOITA/emoita_train_files.csv"):
    print("File already exists. Skipping save to avoid overwriting.")
else:
    emoita_train.to_csv("EMOITA/emoita_train_files.csv", index = False)
    print("File saved successfully.")

File saved successfully.


In [33]:
tmp_dev = pd.read_csv('emozionalmente/1.0.0/fe182b91/db.emotion.categories.dev.desired.csv')
tmp_dev

,file,emotion
0,audio/1614951631538.wav,anger
1,audio/1613994293936.wav,anger
2,audio/1613562322650.wav,anger
3,audio/1614266094803.wav,anger
4,audio/1615229375102.wav,anger
...,...,...
1681,audio/1613210989194.wav,surprise
1682,audio/1613849744545.wav,surprise
1683,audio/1615239203008.wav,surprise
1684,audio/1615228349855.wav,surprise


In [34]:
dev_files = tmp_dev.merge(emoita_raw_files, on="file", how="left")
dev_files.drop(columns=["transcription"], inplace=True)
dev_files

,file,emotion,speaker
0,audio/1614951631538.wav,anger,d4bbc5043503b9aed309ede00854ee48937684b57a1cc0...
1,audio/1613994293936.wav,anger,acdd987f9c1700b25898d3bd30c201df5ad4f34b3ca5eb...
2,audio/1613562322650.wav,anger,4627bdb3ed141168a5e8663c86bfae205566c5bd27779a...
3,audio/1614266094803.wav,anger,07f763295ba2764d034d200456b42eb8bf498d341c586a...
4,audio/1615229375102.wav,anger,253387e8620ee1896c76809782406e139bce35860aaa0d...
...,...,...,...
1681,audio/1613210989194.wav,surprise,de0fdef2b5dea85416a04115cf8c9c1a868dc5ca712ae2...
1682,audio/1613849744545.wav,surprise,d4bbc5043503b9aed309ede00854ee48937684b57a1cc0...
1683,audio/1615239203008.wav,surprise,00a0efc0faaeb5a288481a25b5d0b7d4c0b070120c6d61...
1684,audio/1615228349855.wav,surprise,8e0c2c89852e802de022aa0220dbc78053cfa0a9d8abe2...


In [35]:
dev_files.replace({"speaker": speaker_map}, inplace=True)
dev_files

,file,emotion,speaker
0,audio/1614951631538.wav,anger,speaker_032
1,audio/1613994293936.wav,anger,speaker_139
2,audio/1613562322650.wav,anger,speaker_219
3,audio/1614266094803.wav,anger,speaker_169
4,audio/1615229375102.wav,anger,speaker_161
...,...,...,...
1681,audio/1613210989194.wav,surprise,speaker_201
1682,audio/1613849744545.wav,surprise,speaker_032
1683,audio/1615239203008.wav,surprise,speaker_008
1684,audio/1615228349855.wav,surprise,speaker_358


In [36]:
speaker_dev = set(dev_files["speaker"].unique())
speaker_train = set(emoita_train["speaker"].unique())

if speaker_train.intersection(speaker_dev) == set():
    print("No overlapping speakers between train and dev sets.")
else:
    print("Overlapping speakers found.")
    speaker_train.intersection(speaker_dev)

No overlapping speakers between train and dev sets.


In [37]:
dev_files.rename(columns={"file" : "old_filename"}, inplace=True)

emoita_root = os.path.join("EMOITA", "data")

dev_files["file"] = dev_files["old_filename"].apply(lambda x: os.path.join(emoita_root, os.path.basename(x)))
dev_files

,old_filename,emotion,speaker,file
0,audio/1614951631538.wav,anger,speaker_032,EMOITA/data/1614951631538.wav
1,audio/1613994293936.wav,anger,speaker_139,EMOITA/data/1613994293936.wav
2,audio/1613562322650.wav,anger,speaker_219,EMOITA/data/1613562322650.wav
3,audio/1614266094803.wav,anger,speaker_169,EMOITA/data/1614266094803.wav
4,audio/1615229375102.wav,anger,speaker_161,EMOITA/data/1615229375102.wav
...,...,...,...,...
1681,audio/1613210989194.wav,surprise,speaker_201,EMOITA/data/1613210989194.wav
1682,audio/1613849744545.wav,surprise,speaker_032,EMOITA/data/1613849744545.wav
1683,audio/1615239203008.wav,surprise,speaker_008,EMOITA/data/1615239203008.wav
1684,audio/1615228349855.wav,surprise,speaker_358,EMOITA/data/1615228349855.wav


In [38]:
dev_files =dev_files[["file", "speaker", "emotion"]]
dev_files["split"] = "dev"
dev_files

,file,speaker,emotion,split
0,EMOITA/data/1614951631538.wav,speaker_032,anger,dev
1,EMOITA/data/1613994293936.wav,speaker_139,anger,dev
2,EMOITA/data/1613562322650.wav,speaker_219,anger,dev
3,EMOITA/data/1614266094803.wav,speaker_169,anger,dev
4,EMOITA/data/1615229375102.wav,speaker_161,anger,dev
...,...,...,...,...
1681,EMOITA/data/1613210989194.wav,speaker_201,surprise,dev
1682,EMOITA/data/1613849744545.wav,speaker_032,surprise,dev
1683,EMOITA/data/1615239203008.wav,speaker_008,surprise,dev
1684,EMOITA/data/1615228349855.wav,speaker_358,surprise,dev


In [32]:
if os.path.exists("EMOITA/emoita_dev_files.csv"):
    print("File already exists. Skipping save to avoid overwriting.")
else:
    dev_files.to_csv("EMOITA/emoita_dev_files.csv", index = False)
    print("File saved successfully.")

File saved successfully.


In [39]:
dev_files.to_csv("EMOITA/emoita_dev_files.csv", index = False)

In [40]:
tmp_test = pd.read_csv('emozionalmente/1.0.0/fe182b91/db.emotion.categories.test.desired.csv')
tmp_test

,file,emotion
0,audio/1617135480050.wav,anger
1,audio/1613586149044.wav,anger
2,audio/1617135448748.wav,anger
3,audio/1616505237674.wav,anger
4,audio/1614359087222.wav,anger
...,...,...
1223,audio/1614266389200.wav,surprise
1224,audio/1613821821107.wav,surprise
1225,audio/1614196499838.wav,surprise
1226,audio/1613061195435.wav,surprise


In [41]:
test_files = tmp_test.merge(emoita_raw_files, on="file", how="left")
test_files.drop(columns=["transcription"], inplace=True)
test_files

,file,emotion,speaker
0,audio/1617135480050.wav,anger,28043d2516f2d956b81ce66cc01fbd427ac54ff1eb3a07...
1,audio/1613586149044.wav,anger,0ec8b058931916615ae00bce3d91a997ae440eb9d51ac3...
2,audio/1617135448748.wav,anger,28043d2516f2d956b81ce66cc01fbd427ac54ff1eb3a07...
3,audio/1616505237674.wav,anger,3f9b6ed1d2adc407ef668b8c89a89fe022f2233d94ce94...
4,audio/1614359087222.wav,anger,9299497bc321d5d9ece8f0e6f4932161650ff277c34466...
...,...,...,...
1223,audio/1614266389200.wav,surprise,8c5cae1948feaaa972f368641dedda6eea6e08e853b649...
1224,audio/1613821821107.wav,surprise,f04bb0e6361c05acba4d5185a2d372177bdb77898c1a21...
1225,audio/1614196499838.wav,surprise,b069c4c181f02bf6e0d1b29de3b7e28a4529e315dbb223...
1226,audio/1613061195435.wav,surprise,f4d0c9178a221556efe45a4ea3d2bb77b470626409af7f...


In [42]:
test_files.replace({"speaker": speaker_map}, inplace=True)
test_files

,file,emotion,speaker
0,audio/1617135480050.wav,anger,speaker_413
1,audio/1613586149044.wav,anger,speaker_307
2,audio/1617135448748.wav,anger,speaker_413
3,audio/1616505237674.wav,anger,speaker_168
4,audio/1614359087222.wav,anger,speaker_091
...,...,...,...
1223,audio/1614266389200.wav,surprise,speaker_051
1224,audio/1613821821107.wav,surprise,speaker_001
1225,audio/1614196499838.wav,surprise,speaker_152
1226,audio/1613061195435.wav,surprise,speaker_187


In [43]:
speaker_dev = set(dev_files["speaker"].unique())
speaker_train = set(emoita_train["speaker"].unique())
speaker_test = set(test_files["speaker"].unique())

if speaker_test.intersection(speaker_dev) == set() and speaker_test.intersection(speaker_train) == set():
    print("No overlapping speakers between train or dev and test sets.")
else:
    print("Overlapping speakers found.")
    speaker_train.intersection(speaker_dev)

No overlapping speakers between train or dev and test sets.


In [44]:
test_files.rename(columns={"file" : "old_filename"}, inplace=True)

emoita_root = os.path.join("EMOITA", "data")

test_files["file"] = test_files["old_filename"].apply(lambda x: os.path.join(emoita_root, os.path.basename(x)))
test_files

,old_filename,emotion,speaker,file
0,audio/1617135480050.wav,anger,speaker_413,EMOITA/data/1617135480050.wav
1,audio/1613586149044.wav,anger,speaker_307,EMOITA/data/1613586149044.wav
2,audio/1617135448748.wav,anger,speaker_413,EMOITA/data/1617135448748.wav
3,audio/1616505237674.wav,anger,speaker_168,EMOITA/data/1616505237674.wav
4,audio/1614359087222.wav,anger,speaker_091,EMOITA/data/1614359087222.wav
...,...,...,...,...
1223,audio/1614266389200.wav,surprise,speaker_051,EMOITA/data/1614266389200.wav
1224,audio/1613821821107.wav,surprise,speaker_001,EMOITA/data/1613821821107.wav
1225,audio/1614196499838.wav,surprise,speaker_152,EMOITA/data/1614196499838.wav
1226,audio/1613061195435.wav,surprise,speaker_187,EMOITA/data/1613061195435.wav


In [46]:
test_files =test_files[["file", "speaker", "emotion"]]
test_files["split"] = "test"
test_files

,file,speaker,emotion,split
0,EMOITA/data/1617135480050.wav,speaker_413,anger,test
1,EMOITA/data/1613586149044.wav,speaker_307,anger,test
2,EMOITA/data/1617135448748.wav,speaker_413,anger,test
3,EMOITA/data/1616505237674.wav,speaker_168,anger,test
4,EMOITA/data/1614359087222.wav,speaker_091,anger,test
...,...,...,...,...
1223,EMOITA/data/1614266389200.wav,speaker_051,surprise,test
1224,EMOITA/data/1613821821107.wav,speaker_001,surprise,test
1225,EMOITA/data/1614196499838.wav,speaker_152,surprise,test
1226,EMOITA/data/1613061195435.wav,speaker_187,surprise,test


In [47]:
if os.path.exists("EMOITA/emoita_test_files.csv"):
    print("File already exists. Skipping save to avoid overwriting.")
else:
    test_files.to_csv("EMOITA/emoita_test_files.csv", index = False)
    print("File saved successfully.")

File saved successfully.


In [48]:
emoita_complete_files = pd.concat([emoita_train, dev_files, test_files], ignore_index=True)
emoita_complete_files

,file,speaker,emotion,split
0,EMOITA/data/1613671614352.wav,speaker_321,anger,train
1,EMOITA/data/1613658275427.wav,speaker_303,anger,train
2,EMOITA/data/1613324357435.wav,speaker_314,anger,train
3,EMOITA/data/1614274086698.wav,speaker_109,anger,train
4,EMOITA/data/1612982146424.wav,speaker_179,anger,train
...,...,...,...,...
6897,EMOITA/data/1614266389200.wav,speaker_051,surprise,test
6898,EMOITA/data/1613821821107.wav,speaker_001,surprise,test
6899,EMOITA/data/1614196499838.wav,speaker_152,surprise,test
6900,EMOITA/data/1613061195435.wav,speaker_187,surprise,test


In [49]:
if os.path.exists("EMOITA/emoita_complete_files.csv"):
    print("File already exists. Skipping save to avoid overwriting.")
else:
    test_files.to_csv("EMOITA/emoita_complete_files.csv", index = False)
    print("File saved successfully.")

File saved successfully.
